В данном блокноте рассмотрим простейший пример скрапинга (Scraping) web-cайта средствами Питона. Это один из вариантов получения данных - скачать и распарсить web-cайт.

Получим некий план премьер с сайта https://www.kinopoisk.ru/comingsoon/.

Будем использовать библиотеку Requests, позволяющую отправлять различные http-запросы, и BeautifulSoup, которая помогает извлекать данные из HTML формата.

Импортируем библиотеки.

In [1]:
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import unicodedata


In [3]:
url = 'https://www.kinopoisk.ru/comingsoon/'

Отправляем запрос и получаем html-страницу

In [4]:
html_text = requests.get(url).text
print(html_text)



<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://opengraphprotocol.org/schema/" dir="ltr" lang="ru" prefix="og: http://ogp.me/ns# video: http://ogp.me/ns/video# ya: http://webmaster.yandex.ru/vocabularies/">
<head profile="http://gmpg.org/xfn/11">
    <title>Самые ожидаемые фильмы</title>

    <meta http-equiv="content-type" content="text/html; charset=utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />
    <meta property="fb:app_id" content="121953784483000"/>
    <meta name="application-name" content="Кинопоиск">
    <meta content="width=960" name="viewport" />
    <meta property="fb:pages" content="152308956519" />

    <link rel="search" type="application/opensearchdescription+xml" title="Поиск на kinopoisk.ru" href="/kp_search.xml" />
    <meta name="msapplication-TileColor" content="#000">
    <meta name="msapplication-Ti

Парсим, используя 'lxml' парсер, как наиболее быстрый и простой.

In [5]:
soup = BeautifulSoup(html_text, 'lxml')

Сформируем список всех названий фильмов с web-сайта, используя соответсвующие теги и аттрибуты. Получить информацию о конкретных элементах можно непосредственно в браузере через опцию «Исследовать элемент».

In [6]:
movies_records = []  
movies = soup.find_all('div', class_ = 'info')
for result in movies:  
    name = result.find('div', attrs={'class':'name'}).text
    name = unicodedata.normalize("NFKD",name)
    name = "".join(line.strip() for line in name.split("\n"))
    movies_records.append((name.rsplit(' (',1)[0]))
print(movies_records)


['Сказка для старых', 'Звезды в полдень', 'Вилли Вонка', 'Сердце пармы', 'Шазам! 2', 'МЫ', 'Аквамен и потерянное царство', 'Флэш']


Аналогично сделаем список дат премьер.

In [7]:
movies_dates = []
dates = soup.find_all('div', class_ = 'dateNew')
for result in dates:
  day = result.find('div')["class"][1]
  month = result.find('div', attrs={'class':'month'}).text
  year = result.find('div', attrs={'class':'year'}).text
  movies_dates.append(day.replace('day_','') + " " + month + " " + year)
print(movies_dates)

['13 октября 2022', '01 декабря 2022', '14 декабря 2023', '06 октября 2022', '22 декабря 2022', '01 декабря 2022', '16 марта 2023', '22 июня 2023']


В конце создадим датафрейм с полученной информацией.

In [8]:
df = pd.DataFrame()
df['Название'] = pd.DataFrame(movies_records, columns=['Название'])
df['Дата премьеры'] = pd.DataFrame(movies_dates, columns=['Дата премьеры'])
print(df)

                       Название    Дата премьеры
0             Сказка для старых  13 октября 2022
1              Звезды в полдень  01 декабря 2022
2                   Вилли Вонка  14 декабря 2023
3                  Сердце пармы  06 октября 2022
4                      Шазам! 2  22 декабря 2022
5                            МЫ  01 декабря 2022
6  Аквамен и потерянное царство    16 марта 2023
7                          Флэш     22 июня 2023
